# Running tasks parallelly and distributedly

## Motivation

Take the following DAG structure. We have two upstream tasks that then fork out into three independent tasks.

![img](https://miro.medium.com/max/1400/1*Y4W6pHkPQbczomnhR7PZdQ.jpeg)

If each of the three tasks takes 10 minutes to run, then running them sequentially will take 30 minutes. But if we have enough compute resources, we can just run them in parallel instead and reduce the execution time by 2/3rds. When tasks are independent, we can run them concurrently.

Note that the independent tasks do not have to be homogenous tasks. They can be entirely different functions as long as they are independent.

This section will cover how to execute tasks parallelly or distributedly.

Before talking about how to parallelize, we just need to clarify two terms that will be used a lot.

## Parallel Computing versus Distributed Computing

[This](https://www.oreilly.com/library/view/distributed-computing-in/9781787126992/7478b64c-8de4-4db3-b473-66e1d1fcba77.xhtml) O'Reilly page describes the different of parallel and distributed computing. Parallel computing normnally is confised to one machine where the processes can share the same memory, while distributed computing happens over multiple machines. In the distributed scenario, a task and the dependencies are sent to other machines through a network connection.

![img](https://www.oreilly.com/library/view/distributed-computing-in/9781787126992/assets/e1153739-f551-4f1f-a44a-8effcd193099.png)

With that, we can introduce Dask, a distributed comnputing framework that allows us to scale workflows. Something convenient about Dask is that the local mode is similar to a multiprocessing pool while the distributed mode executes code across a cluster.

## Introduction to Dask

[Dask](https://dask.org/) is an open-source distributed computing framework built to extend the PyData stack. [This issue](https://github.com/dask/dask/issues/4471) on the Dask repo has some architecture diagrams that illustrate Dask's architecture.

Most commonly, people are introduced to Dask as a means to scale Pandas. Pandas can be inefficient because a lot of Pandas operations generate [intermediate copies](https://pandas.pydata.org/pandas-docs/stable/user_guide/scale.html#scaling-to-large-datasets) of data, utilizing more memory than necessary. To effectively handle data with pandas, users preferably need to have [5x to 10x times](https://wesmckinney.com/blog/apache-arrow-pandas-internals/) as much RAM as the size of the dataset. 

Dask scales Pandas by creating a higher level of Pandas DataFrames called the Dask DataFrame. The image below shows this architecture. Each Pandas DataFrame here is referred to as a partition, which is a logical (and physical) grouping of data.

![img](https://user-images.githubusercontent.com/306380/129031375-83547ea2-b3fd-4623-ad9a-e57ddc23a9e6.jpg)

We can see example Dask DataFrame code below, notice the wildcard shows we can load multiple files in parallel. The API was also designed to mirror the Pandas API.
```python
import dask.dataframe as dd
df = dd.read_csv('2014-*.csv')
df.head()
```

Under the hood, Dask DataFrame then takes care of submitting this for execution to the Dask cluster. 

![img](https://user-images.githubusercontent.com/306380/129031416-fe117b62-83f6-47ce-9227-ba8a50db3bf8.jpg)

The Client can give on your local computer, and can submit tasks to the Dask cluster. The Schedeuler is the entrypoint that receives this task and decides which worker to send it to. When using the Dask DataFrame API, Dask handles the lower level managing of sending partitions to workers or rearranging the data across the cluster. This is called a shuffle in distributed compute terms.

## Memory Bound versus Compute Bound Problems

The [Dask-ml](https://ml.dask.org/) has a good introduction to the cases that need distributed compute. They either tend to be compute-bound problems or memory-bound problems. For memory-bound problems, the data literally does not fit on a single machine so we need to divide across multiple machinses and keep trac k where each partition of data lives.

On the other hand, compute-bound problems are about tasks that take long to execute to we can speed up execution by running them in parallel or distributedly over a cluster. The point is to utilize untapped resoucres that are not consumed when executing sequentially.

![img](https://ml.dask.org/_images/dimensions_of_scale.svg) 

## Dask collections

But while we have discussed this in the context of DataFrames, the DataFrame concept is just one of the available collections. In fact, it is a collection more associated with memory-bound problems while workflow orchestration usage of Dask tends to be compute-based problems.

The diagram below shows the other available collections.

![img](https://user-images.githubusercontent.com/306380/129031388-6fcd0cd1-9643-4f3d-b4b1-c6e343bbcf08.jpg)

The Dask Bag is like a distributed dictionary or JSON. The Dask Array builds on top of `xarray`. But the collection that is most relevant to workflow orchestration is Futures.

## Dask Futures

Dask Delayed and Dask Futures allow for the submission of arbitrary code to the Dask scheduler. The Dask scheduler then directs the execution to a worker. The difference is that Dask Delayed is evaluated lazily, allowing the computation graph to compile before execution. Knowing the execution graph ahead of time allows Dask to optimize it by analyzing the data depdenencies and ensuring that workers have all the dependencies they need to execute tasks. Dask Futures, on the other hand are executed immediately.

Below is an example of using the Future interface, which is what Prefect does to submit tasks to a Dask cluster. Here, we perform the execution on a local cluster.

In [3]:
from dask.distributed import Client

client = Client(processes=False)

def inc(x):
    return x + 1

a = client.submit(inc, 10)
b = client.submit(inc, 20)
print(a)
print(a.result())

<Future: pending, key: inc-63cff39c2c9137a6b939de9f3426d97b>
11


/opt/miniconda3/envs/orion/lib/python3.8/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 49869 instead
  warnings.warn(


## When to use Dask versus Prefect

People already familiar with Dask wonder what Prefect adds because Dask already seemingly has a lot of the features Prefect brings. For example, Dask already has

* Retries of code
* Scheduling of tasks
* A Directed Acyclic Graph (DAG)
* Handling of dependencies

So when do we use Dask versus Prefect? Or how are they used together?

**The Prefect DAG**

The Prefect DAG is more robust in handling state. For example, it is superior in the following cases:

* restarting a failed Flow from the checkpoint
* caching across multiple flow runs
* linking results to Flow runs (and parameters) through the GraphQL API
* observability into why a Flow failed (additional logging)

So what is the relationship of the Dask DAG and the Prefect DAG? Prefect is a more macro-level workflow orchestrator while Dask focuses on compute (micro-level). Prefect cares more across Flow runs while you would need to add extra code for Dask to Handle it.

## Mapping in Prefect 1.0 (Depth First Execution)

There is actually an edge case when the Dask DAG is used. This is when we have multiple stages of mapping. For example, check the two-stage mapping operation below. By submitting multiple stages to Dask together, we can execute them in a depth-first fashion. This eliminates the need to collect intermediate results before running the next task. 

![img](https://miro.medium.com/max/602/1*KOi-kSaQdDIdTSGhaOGenA.png)

When these are submitted together, the Dask DAG is used (and sometimes overlaps with the Prefect DAG). For example, if a Dask worker dies, the different stages can be executed again (sometimes duplicated). **This is because Dask revives the worker and from it's perspective, does not realize that earlier tasks have been completed and the checkpoint can be used.**

## Setting-up a Dask cluster

One reason why Dask is widely adopted is because of the ease of spinning up your own ephemeral cluster for the duration of an application (or flow run). [This documentation page](https://docs.dask.org/en/stable/deploying.html) contains the various ways that a Dask cluster can be deployed. The choice of cluster largely depeneds on your infrastructure. The most commons ones are:

* Dask on Kubernetes
* Dask on AWS/ECS or Fargate
* Managed services like [Coiled](https://coiled.io/) or [Saturn Cloud](https://saturncloud.io/)

The important thing to note in distributed systems is that the package versions on the workers need to be the same as the scheduler and client. Otherwise, it's very easy to run into inconsistent execution, or programs will raise an error. Most Dask cluster initialization method will take in a Docker base image that can be used to spin-up the workers. This guarantees execution.

For example, this is what spinning up a Dask cluster looks like with a `KubeCluster`

```python
from dask_kubernetes import KubeCluster, make_pod_spec

pod_spec = make_pod_spec(image='prefecthq/prefect:latest')
cluster = KubeCluster(pod_spec)
cluster.scale(10)
```
**This is why it's important to know how to build your own image to include the dependencies**

After this, you can connect the Client with:
```
client = Client(cluster)
```
but Prefect does it for you under the hood in order to submit your tasks.

## Dask versus other distributed computing frameworks

When Prefect started out, it specifically was exploring performing Airflow-like workflow orchestration semantics on top of the Dask milli-second latency scheduler. This is because the Airflow scheduler took 10 seconds to submit a task. But why Dask specifically compared to Spark?

Spark and Dask are the most widely adopted distributed compute interfaces for Python. The difference though is that Spark has leaned more heavily towards the DataFrame abstraction and has optimized for that. It's very easy in Dask to submit an abitrary function with

```python
client.submit(fn)
```

whereas in Spark it's not even straightforward how to perform this operation. In [this](https://databricks.com/blog/2016/07/14/a-tale-of-three-apache-spark-apis-rdds-dataframes-and-datasets.html) blog by Databricks, they recommend using the DataFrame API. 

Ray, though has been gaining steam as a distributed computing framework so we added a task runner for it in Orion. The docs can be [found here](https://orion-docs.prefect.io/tutorials/dask-ray-task-runners/). The benefits of Ray over Dask were discussed [here](https://github.com/PrefectHQ/prefect/issues/3963). Namely (mentioned by Github user Hoeze):

* Ray would make it easy to define resources that a certain task needs (e.g. GPU, memory):
* Also, it's shared-memory object store is just superior to Dask's deserialization for large objects when working with Dask tasks (e.g. check [dask-on-ray](https://github.com/ray-project/ray/issues/13620) Zero-copy deserialization of dask.delayed() objects.

## Using the Dask TaskRunner for Orion

Utilizing the `DaskTaskRunner` in Prefec Orion is as simple as adding it to the `@flow` decorator. If not specified, the default Task Runner used is the `ConcurrentTaskRunner`, which actually runs tasks asynchronously already. It should be comparable to Dask on local, and now Dask can scale beyond one machine. 

The `ConcurrentTaskRunner` was introduced to remove the Dask dependency for local parallelism.

In [ ]:
from prefect import flow, task
from prefect.task_runners import DaskTaskRunner

@task
def say_hello(name):
    print(f"hello {name}")

@task
def say_goodbye(name):
    print(f"goodbye {name}")

@flow(task_runner=DaskTaskRunner())
def greetings(names):
    for name in names:
        say_hello(name)
        say_goodbye(name)

if __name__ == "__main__":
    greetings(["arthur", "trillian", "ford", "marvin"])

We can then pass a callable or address to the `DaskTaskRunner` to create an ephemeral cluster or connec to an already existing cluster.

```python
DaskTaskRunner(
    cluster_class="dask_cloudprovider.FargateCluster",
    cluster_kwargs={
         "image": "prefecthq/prefect:latest",
         "n_workers": 5,
    },
)
```

Note we can also do the same for Ray. In the next section, we'll see how to create our own image that we can use to guarantee consistency across Flow Runs and when creating a cluster.